<a href="https://colab.research.google.com/github/aims-ks/transformer-tutorials/blob/main/Calculate_reefscape_normalisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up environment

Let's start by installing our favorite libraries: 🤗 Transformers and 🤗 Datasets. We install Transformers from source since the model is brand new at the time of writing.

In [2]:
!pip install -q git+https://github.com/huggingface/transformers.git datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.8 MB/s eta 0:00:00


In this notebook we calculate the mean and standard deviation of the reefscapes marine features dataset. This is metadata that is used to normalise the images.

In [3]:
import numpy as np
import torch
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("aimsks/ts-aims-reefscapes-marine-features")

# Convert RGBA images to RGB tensors and normalize to 0-1 range

def image_to_tensor(image):
    # Convert the image to a tensor, keep only the RGB channels, and normalize
    return torch.tensor(np.array(image)[:,:,:3]).permute(2, 0, 1).float() / 255.0

# Calculate the mean and std for the training set
mean = torch.zeros(3)
std = torch.zeros(3)

print("Calculating mean")
for data in dataset["train"]:
    image = image_to_tensor(data["image"])
    mean += image.mean([1, 2])

mean /= len(dataset["train"])

print("Calculating std")
for data in dataset["train"]:
    image = image_to_tensor(data["image"])
    std += ((image - mean[:, None, None]) ** 2).mean([1, 2])

std = torch.sqrt(std / len(dataset["train"]))

print("Mean (RGB):", mean)
print("Std (RGB):", std)


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/772 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/194 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/423 [00:00<?, ? examples/s]

Calculating mean
Calculating std
Mean (RGB): tensor([0.1839, 0.3785, 0.4397])
Std (RGB): tensor([0.1389, 0.1073, 0.0956])
